In [3]:
import requests
import os
import pandas as pd
from IPython.core.display import HTML
import json
from cred import credentials

In [57]:
# to connect to the Genesis API via data/table endpoint
de_request_url = f'https://www-genesis.destatis.de/genesisWS/rest/2020/data/table'
de_request_params = {
    'username': username,
    'password': password,
    "name": "12411-0001",
    "area": "all",
    "compress": "false",
    "transpose": "false",
    "startyear": "",
    "endyear": "",
    "timeslices": "",
    "regionalvariable": "",
    "regionalkey": "",
    "classifyingvariable1": "",
    "classifyingkey1": "",
    "classifyingvariable2": "",
    "classifyingkey2": "",
    "classifyingvariable3": "",
    "classifyingkey3": "",
    "job": "false",
    "stand": "01.01.1970",
    "language": "en" ,
}
# save the response
de_response = requests.get(url=de_request_url, params=de_request_params)
# to save the response as json in a file called "r_population.json" 
with open('data/r_population.json', 'w') as f:
    # json.dump(de_response.json(), f, indent=2)
    f.write(json.dumps(de_response.json(), indent=2))
# to save the part of response (the table) as a csv called "r_population.csv" 
with open('data/r_population.csv', 'w') as csv_file:
    csv_file.write(de_response.json()['Object']['Content'])

In [31]:
# define a function to request the Genesis API 
def get_genesis_api(
        code: str,
        years: list[int],
        skiprows: int=4,
        skipfooter: int=4,
        delimiter: str=';',
        header: list[int]=[0,1],
        save_json: bool=False,
) -> pd.DataFrame:

    dataframes = {}
    for year in years:
        de_request_url = f'https://www-genesis.destatis.de/genesisWS/rest/2020/data/table'
        de_request_params = {
            'username': credentials['username'],
            'password': credentials['password'],
            "name": code,  # e.g. "12411-0014" the population per state, sex, nationality
            "area": "all",
            "compress": "false",
            "transpose": "false",
            "startyear": year,
            "endyear": year,
            "timeslices": "",
            "regionalvariable": "",
            "regionalkey": "",
            "classifyingvariable1": "",
            "classifyingkey1": "",
            "classifyingvariable2": "",
            "classifyingkey2": "",
            "classifyingvariable3": "",
            "classifyingkey3": "",
            "job": "false",
            "stand": "01.01.1970",
            "language": "en" ,
        }
        # to save the response as json in a file called "r_population_kries_year.json"
        de_response = requests.get(url=de_request_url, params=de_request_params)
        if save_json:
            with open(f'data/{code}_{year}.json', 'w') as f:
                json.dump(de_response.json(), f, indent=2)

        # to save the part of response (the table) as a csv called "r_population_kreis_year.csv" 
        with open(f'data/{code}_{year}.csv', 'w') as csv_file:
            csv_file.write(de_response.json()['Object']['Content'])

        dataframes[year] = pd.read_csv(f'data/{code}_{year}.csv', skiprows=skiprows, skipfooter=skipfooter, delimiter=delimiter, header=header)  
    
    result = pd.concat(dataframes.values(), axis=0)
    result.to_csv(f'data/{code}_{min(years)}_{max(years)}.csv')
    return result



In [ ]:
get_genesis_api('12411-0012', [2012, 2014])